In [1]:
import sys
sys.path.append('../bfrt_grpc')
from bfrt_grpc_helper import BfRtAPI
from bfrt_grpc_client import Bfrt_GRPC_Client
import time
import bfrt_grpc.client as gc
import bfrt_grpc.bfruntime_pb2 as bfruntime_pb2

In [2]:
remote_grpc_addr = '192.168.24.69:50052'
controller1 = BfRtAPI(remote_grpc_addr, client_id=0, perform_subscribe=False, perform_bind=None)

Received tf_honeypot on GetForwarding on client 0, device 0


In [7]:
bfrt_info = controller1.interface.bfrt_info_get()
target = gc.Target(device_id=0)

service_table = bfrt_info.table_get("pipe.Ingress.active_host_tbl")
service_table.info.key_field_annotation_add("meta.internal_ip", "ipv4")


Received tf_honeypot on GetForwarding on client 0, device 0


In [8]:
key_list = [service_table.make_key([gc.KeyTuple('meta.internal_ip', 783663009),
                                    gc.KeyTuple('meta.internal_port', 10),
                                    gc.KeyTuple('meta.ip_protocol', 6)])]
data_list = [service_table.make_data([], 'Ingress.drop')]
service_table.entry_add(target, key_list, data_list, p4_name='tf_honeypot')


In [3]:
controller1.clear_table('pipe.Ingress.active_host_tbl')

Problem clearing pipe.Ingress.active_host_tbl


In [ ]:
# service_table = controller1.bfrt_info.table_get('active_host_tbl')
# service_table.entry_del(gc.Target(), [])
controller1.entry_add(table_name='pipe.Ingress.active_host_tbl', 
                                keys=[('meta.internal_ip', 783663009),
                                    ('meta.internal_port', 10),
                                    ('meta.ip_protocol', 6)],
                                data=[],
                                action_name='Ingress.drop')

Error(s):



In [4]:
notf = gc.Notifications(enable_idletimeout=True, enable_entry_active=False, enable_port_status_change=False, enable_learn=False)
controller2 = BfRtAPI(remote_grpc_addr, client_id=1, perform_subscribe=True, perform_bind=True, notifications=notf)

Subscribe attempt #1
Subscribe response received 0
Received tf_honeypot on GetForwarding on client 1, device 0
Binding with p4_name tf_honeypot
Binding with p4_name tf_honeypot successful!!


In [5]:
controller2.clear_table('pipe.Ingress.active_host_tbl')

In [5]:
controller2.print_tables_info()

Full Table Name                    Type                Usage      Capacity
---------------------------------  ------------------  -------  ----------
pipe.Ingress.active_host_tbl       MatchAction_Direct  71664        500000
pipe.Ingress.internal_ip_check     MatchAction_Direct  2                10
pipe.Ingress.mirror_fwd            MatchAction_Direct  1               512
pipe.Ingress.whitelist_tbl         MatchAction_Direct  10              512
pipe.snapshot.cfg                  SnapshotCfg         0                12
pipe.snapshot.egress_trigger       SnapshotTrigger     n/a              12
pipe.snapshot.egress_data          SnapshotData        12               12
pipe.snapshot.egress_liveness      SnapshotLiveness    0                 0
pipe.snapshot.ingress_trigger      SnapshotTrigger     n/a              12
pipe.snapshot.ingress_data         SnapshotData        12               12
pipe.snapshot.ingress_liveness     SnapshotLiveness    0                 0
pipe.IngressParser.$PORT_